In [3]:
from Tkinter import *
from __future__ import division
import tkFileDialog
from math import cos, pi, fabs, tan
from numpy import sign,arange, sin, array
from numpy.linalg import norm
from test_slam3 import Particle, FastSLAM
from boundaries import boundaries
from walls import walls
from Sphero_test3 import Sphero
import matplotlib as mpl
from time import time
import copy
import sys
import matplotlib.pyplot as plt
import book_plot as bp
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg

def start():
    global pose, pose1, control, constraints, loop, radius, wall_width, world_size, flag, number_of_particles
    scale = 520
    loop = not(loop)
    pmeasurement, pmeasurement1 = [], []
    wall, wall1 = [], []
    weights = [1/number_of_particles]*number_of_particles
    particle_pose = [pose]*number_of_particles
    particle_pose1= [pose1]*number_of_particles
    ball_pose = pose  # The simulation of ball movement is carried out separately
    ball_pose1= pose1
    i=0
    set_merge = 1
    while loop and i<=5000:
        past_orient = []
        past_orient1= []
        
        ball_ppose = ball_pose  # Ball simulation
        ball_ppose1= ball_pose1
        
        particle_ppose = particle_pose # Estimated ball pose
        particle_ppose1= particle_pose1
        
        # Moving the ball
        ball_pose = s.motion(ball_ppose,control,2,0.01)  # Ball motion, which is same as in the real case
        ball_pose1 = s1.motion(ball_ppose1,control,2,0.01)
        
        particle_pose = fs.predict(control)   # pf prediction of the robot
        particle_pose1= fs1.predict(control)
        
        past_orient = ball_pose[2] # Past orientation of robot to find the measurement
        past_orient1= ball_pose1[2]

        # Check for measurement; measurement to be included as prior
        ball_pose = s.check_collision(constraints, ball_pose, ball_ppose, flag)# flag = wall_correction
        ball_pose1= s1.check_collision(constraints, ball_pose1, ball_ppose1, flag)

        # Draw the robot which includes the effect of wall correction
        f=s.draw(ball_pose[0])
        g=s1.draw(ball_pose1[0])
        
        # pf correction step if measurement has occured
        if ball_pose[1] != -1: 
            
            # Correcting the particles for orientation;fsm - This is the continuation of prediction step of pf
            particle_pose = s.orientation_correction(particle_pose, ball_pose[1]) 
            
            # Mesurement from ball
            measurement = array([ball_pose[0][0],ball_pose[0][1],tan(ball_pose[1]-past_orient)]) 
            
            # pf correction step; includes the pose before the orientation correction
            weights, wall, lw = fs.correct(measurement,pmeasurement)
            
            # Drawing the map
            for j in range(len(wall)):
                if j==2 or j==4:
                    wall[j] = list(reversed(wall[j]))
            robot_map(wall)
            pmeasurement = measurement
        
        if ball_pose1[1] != -1: 
            
            # Correcting the particles for orientation;fsm - This is the continuation of prediction step of pf
            particle_pose1 = s1.orientation_correction(particle_pose1, ball_pose1[1]) 
            
            # Mesurement from ball
            measurement1 = array([ball_pose1[0][0],ball_pose1[0][1],tan(ball_pose1[1]-past_orient1)]) 
            
            # pf correction step; includes the pose before the orientation correction
            weights1, wall1, lw1 = fs1.correct(measurement1,pmeasurement1)
            
            # Drawing the map
            for j in range(len(wall1)):
                if j==2 or j==4:
                    wall1[j] = list(reversed(wall1[j]))
            robot_map1(wall1)
            pmeasurement1 = measurement1
        
        if set_merge == 0 and i==300:
            robot_map2(wall,wall1,lw,lw1)
            set_merge = 2
        
        root.update()
        i = i + 1
        ball_pose = ball_pose[0]
        ball_pose1= ball_pose1[0]
        if len(wall) >=5 and len(wall1) >= 5 and set_merge == 1:
#             merge_map(wall,wall1,lw,lw1)
            set_merge = 0

def stop():
    global loop
    loop = not(loop)

def boost():
    global control
    control[0] = control[0] + 50
    if control[0] <= 0:
        control[0] =0

def slow():
    global control
    control[0] = control[0] - 50
    if control[0] <= 0:
        control[0] =0

def correct_wall():
    global pose, control, loop, flag
    loop = not(loop)
    pose = [100,150,pi/4]
    control = [100,0]
    loop = 0
    flag = 1

# The boundary of the robot is assumed to be known and the robots start from the end points    
# Delayed data association to be included since not much features are added.
def merge_map(wall,wall1,lw,lw1):
    for i in range(4):
        a = lw[i]
        b = lw1[i]
        if fabs(a[1][0] - b[1][0]) >= 450:
            # Find the orientation difference between these two edges and rotate (translate also if needed) the maps to merge them
            if a[1][1] == b[1][1]:
                # The end walls have the same orientation
                orientation_diff = 0
                score = norm((array(wall1[i])-array(wall[i])),ord=1)
            else:
                orientation_diff = pi
                # The end walls have opposite orientation
                score = norm((array(list(reversed(wall1[i])))-array(wall[i])),ord=1)
            robot_map2(wall,wall1,lw,lw1)
        
def robot_map(wall):
    a= [[]]
    for i in range(len(wall)):
        wall[i] = [(1-wall[i][k]/max(wall[i])) for k in range(len(wall[i]))]
        wall[i] = list(reversed(wall[i]))
        for j in range(len(wall[i])):
            a[i].append([wall[i][j]]*3)
        a.append([])
        
    
    for i in range(len(a)-1): # Do not plot the empty list in the last of 'a' due to the above operation 
        if i == 0:
            ax1 = fig1.add_axes([0.25, 0.01, 0.025, 0.98]) # GUI rectangle creation
            ax1.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[0])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
#             plt.gcf().canvas.draw()
            plt.draw()
            
        elif i == 1:
            ax1 = fig1.add_axes([0.01, 0.01, 0.025, 0.98]) # GUI rectangle creation
            ax1.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[1])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
#             plt.gcf().canvas.draw()
            plt.draw()
            
        elif i == 2:
            ax1 = fig1.add_axes([0.01, 0.01, 0.98, 0.025]) # GUI rectangle creation
            ax1.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[2])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                               norm=norm,
                                               orientation='horizontal')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
            
        elif i == 3:
            ax1 = fig1.add_axes([0.5, 0.01, 0.025, 0.98]) # GUI rectangle creation
            ax1.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[3])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
            
        elif i == 4:
            ax1 = fig1.add_axes([0.01, 0.965, 0.98, 0.025]) # GUI rectangle creation
            ax1.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[4])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                               norm=norm,
                                               orientation='horizontal')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
            
        elif i == 5:
            ax1 = fig1.add_axes([0.74, 0.01, 0.025, 0.98]) # GUI rectangle creation
            ax1.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[5])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
        
        elif i == 6:
            ax1 = fig1.add_axes([0.967, 0.01, 0.025, 0.98]) # GUI rectangle creation
            ax1.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[6])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
    
def robot_map1(wall):
    a= [[]]
    for i in range(len(wall)):
        wall[i] = [(1-wall[i][k]/(max(wall[i]))) for k in range(len(wall[i]))]
        wall[i] = list(reversed(wall[i]))
        for j in range(len(wall[i])):
            a[i].append([wall[i][j], 0, 0])
        a.append([])
        
    
    for i in range(len(a)-1): # Do not plot the empty list in the last of 'a' due to the above operation 
        if i == 0:
            ax2 = fig2.add_axes([0.74, 0.01, 0.025, 0.98]) # GUI rectangle creation [0.74, 0.01, 0.025, 0.98]
            ax2.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[0])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
#             plt.gcf().canvas.draw()
            plt.draw()
            
        elif i == 1:
            ax2 = fig2.add_axes([0.967, 0.01, 0.025, 0.98]) # GUI rectangle creation [0.967, 0.01, 0.025, 0.98]
            ax2.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[1])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
#             plt.gcf().canvas.draw()
            plt.draw()
            
        elif i == 2:
            ax2 = fig2.add_axes([0.01, 0.01, 0.98, 0.025]) # GUI rectangle creation
            ax2.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[2])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap1,
                                               norm=norm,
                                               orientation='horizontal')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
            
        elif i == 3:
            ax2 = fig2.add_axes([0.5, 0.01, 0.025, 0.98]) # GUI rectangle creation
            ax2.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[3])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
            
        elif i == 4:
            ax2 = fig2.add_axes([0.01, 0.965, 0.98, 0.025]) # GUI rectangle creation
            ax2.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[4])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap1,
                                               norm=norm,
                                               orientation='horizontal')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
            
        elif i == 5:
            ax2 = fig2.add_axes([0.25, 0.01, 0.025, 0.98]) # GUI rectangle creation [0.01, 0.01, 0.025, 0.98]
            ax2.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[5])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
        
        elif i == 6:
            ax2 = fig2.add_axes([0.01, 0.01, 0.025, 0.98]) # GUI rectangle creation [0.25, 0.01, 0.025, 0.98]
            ax2.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[6])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()

def robot_map2(wall,wall1,lw,lw1):
    a= [[]]
    for i in range(5):
#         wall[i] = [(1-wall[i][k]/(max(wall[i]))) for k in range(len(wall[i]))]
#         wall[i] = list(reversed(wall[i]))
        for j in range(len(wall[i])):
            a[i].append([wall[i][j]]*3)
        a.append([])
    b = [[]]    
    for i in range(5):
#         wall1[i] = [(1-wall1[i][k]/(max(wall1[i]))) for k in range(len(wall1[i]))]
#         wall1[i] = list(reversed(wall1[i]))
        for j in range(len(wall1[i])):
            b[i].append([wall1[i][j], 0, 0])
        b.append([])
        
    
    for i in range(len(a)-1): # Do not plot the empty list in the last of 'a' due to the above operation 
        if i == 0:
            ax = fig3.add_axes([0.25, 0.01, 0.025, 0.98]) # GUI rectangle creation [0.74, 0.01, 0.025, 0.98]
            ax.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[0])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
            
        elif i == 1:
            ax = fig3.add_axes([0.01, 0.01, 0.025, 0.98]) # GUI rectangle creation [0.967, 0.01, 0.025, 0.98]
            ax.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[1])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
            
        elif i == 2:
            ax = fig3.add_axes([0.01, 0.01, 0.98, 0.025]) # GUI rectangle creation
            ax.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[2])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap1,
                                               norm=norm,
                                               orientation='horizontal')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
            
        elif i == 3:
            ax = fig3.add_axes([0.5, 0.01, 0.025, 0.98]) # GUI rectangle creation
            ax.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[3])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
            
        elif i == 4:
            ax = fig3.add_axes([0.01, 0.965, 0.98, 0.025]) # GUI rectangle creation
            ax.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(a[4])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap1,
                                               norm=norm,
                                               orientation='horizontal')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
        
    for i in range(len(b)-1):
        if i == 0:
            ax = fig3.add_axes([0.74, 0.01, 0.025, 0.98]) # GUI rectangle creation [0.01, 0.01, 0.025, 0.98]
            ax.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(b[0])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
        
        elif i == 1:
            ax = fig3.add_axes([0.967, 0.01, 0.025, 0.98]) # GUI rectangle creation [0.25, 0.01, 0.025, 0.98]
            ax.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(b[1])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
        
        elif i == 2:
            ax = fig3.add_axes([0.01, 0.01, 0.98, 0.025]) # GUI rectangle creation
            ax.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(b[2])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap1,
                                               norm=norm,
                                               orientation='horizontal')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
        
        elif i == 3:
            ax = fig3.add_axes([0.5, 0.01, 0.025, 0.98]) # GUI rectangle creation
            ax.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(b[3])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap1,
                                               norm=norm,
                                               orientation='vertical')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
            
        elif i == 4:
            ax = fig3.add_axes([0.01, 0.965, 0.98, 0.025]) # GUI rectangle creation
            ax.set_axis_off()
            cmap1 = mpl.colors.ListedColormap(b[4])
            cmap2 = mpl.cm.Greys
            norm = mpl.colors.Normalize()
            cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap1,
                                               norm=norm,
                                               orientation='horizontal')
            cb1.outline.set_visible(False)
            plt.draw()
#             plt.gcf().canvas.draw()
        
    
if __name__ =="__main__":

    # Specifications of the world
    origin = (2,2)
    world_size = (498, 498)   # 750,750
    wall_width = 10
    radius = 10
    plotShift = 0
    flag = 0 # For wall correction
    wall_location =[[origin[0]+(world_size[0])/4,origin[1]+wall_width,origin[0]+(world_size[0])/4,origin[1]+wall_width+(world_size[1])*3/4],
                    [origin[0]+(world_size[0])/2,origin[1]+wall_width+(world_size[1])*1/4,origin[0]+(world_size[0])/2,origin[1]-wall_width+world_size[1]],
                    [origin[0]+(world_size[0])*3/4,origin[1]+wall_width,origin[0]+(world_size[0])*3/4,origin[1]+wall_width+(world_size[1])*3/4]] 
    
    #Initialization of boundary and walls
    sri = boundaries(world_size,wall_width) 
    mw = walls(wall_width,wall_location)
    
    # Intialization of GUI
    root = Tk()
    root.title("Ball in a maze")
    frame1 = Frame(root)
    frame1.pack()
    w = Canvas(frame1,width=500,height=500,bg="white")
    w.pack(side=LEFT)
    
    # Robot map
    r = Canvas(frame1,width=500,height=500,bg="white")
    r.pack(side=RIGHT)
    fig = plt.figure(figsize=(6.15,6.15),facecolor='w',edgecolor='w')
    canvas = FigureCanvasTkAgg(fig, master=r)
    canvas.get_tk_widget().pack(side=RIGHT)
    
    fig1 = plt.figure(figsize=(6.15,6.15),facecolor='w',edgecolor='w')
    fig1.canvas.set_window_title('Robot 1')
#     fig1.show()
    
    fig2 = plt.figure(figsize=(6.15,6.15),facecolor='w',edgecolor='w')
    fig2.canvas.set_window_title('Robot 2')
#     fig2.show()
    
    fig3 = plt.figure(figsize=(6.15,6.15),facecolor='w',edgecolor='w')
    fig3.canvas.set_window_title('Merged Map')
    fig3.show()
    
    # Creating world objects
    sri.draw(origin,w)
    mw.draw(wall_width,wall_location,w)
    parray = sri.boundary_grid(origin,w)
    warray = mw.wall_grid(wall_width,wall_location,w)
    
    # Creating Sphero robot
    s = Sphero(w)
    s1 = Sphero(w)
    
    constraints = s.sphero_constraint(origin,wall_width,world_size,wall_location)

    minimum_correspondence_likelihood = 1e-3
    xstddev = 0.001
    ystddev = 0.001
    measurement_stddev = 0.001
    control_speed_factor = 0.01
    control_head_factor = 0.01
    number_of_particles = 10
    robot_width = 2
    sample_time = 0.01
    
    pose = [100,150,pi/4]
    pose1 = [460,150,3*pi/4]
    control = [100,0]
    loop = 0
    
    initial_particles = [copy.copy(Particle(pose))
                         for _ in xrange(number_of_particles)]
    initial_particles1 = [copy.copy(Particle(pose1))
                         for _ in xrange(number_of_particles)]
    
    fs = FastSLAM(initial_particles,robot_width,minimum_correspondence_likelihood,measurement_stddev,xstddev,ystddev,
                 control_speed_factor,control_head_factor, sample_time)
    fs1 = FastSLAM(initial_particles1,robot_width,minimum_correspondence_likelihood,measurement_stddev,xstddev,ystddev,
                 control_speed_factor,control_head_factor, sample_time)
    
    frame2 = Frame(root)
    frame2.pack()
    start = Button(frame2,text="START", width=10,command=start)
    start.grid(row=0,column=0)
    stop = Button(frame2,text="STOP",width=10,command=stop)
    stop.grid(row=0,column=1)
    boost = Button(frame2,text="BOOST", width=10,command=boost)
    boost.grid(row=0,column=2)
    slow = Button(frame2,text="SLOW", width=10,command=slow)
    slow.grid(row=0,column=3)
    wc = Button(frame2,text="Wall Correction", width=15,command=correct_wall)
    wc.grid(row=0,column=4)
    
    # Draw GUI
    root.mainloop()

Entered
i is 500
First
Second


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Srinivas\Anaconda\lib\lib-tk\Tkinter.py", line 1486, in __call__
    return self.func(*args)
  File "<ipython-input-3-deb46cc5b67b>", line 57, in start
    f=s.draw(ball_pose[0])
  File "Sphero_test3.py", line 36, in draw
    map(self.canvas.delete, self.cursor_objects)
  File "C:\Users\Srinivas\Anaconda\lib\lib-tk\Tkinter.py", line 2304, in delete
    self.tk.call((self._w, 'delete') + args)
TclError: invalid command name ".320283272L.320283336L"
